# CtF-Game Policy Tournament

- Simple script to test the performance of individual policy.
- Graph the success reward, and episode length.
- Script __does not__ include the rendering

## TODO:


In [10]:
import os

import time
import gym
import gym_cap
import numpy as np

from collections import defaultdict

# the modules that you can use to generate the policy.
import policy.zeros
import policy.patrol # only for non-stochastic zone? 
import policy.random
import policy.roomba # Supposed to be heuristic
import policy.policy_RL # RL weights are in ./model directory

## Policy/Model Selection

In [11]:
#rl_model = ['./model/'+name for name in os.listdir('./model')]
rl_model = ['./model/'+name for name in ('B4R4_Rzero_VANILLA', 'B4R4_self_VANILLA')]
rl_nick = ['red-zero train', 'self-play']

## Training Settings

In [12]:
env = gym.make("cap-v0") # initialize the environment

n_episode   = 1000;
ep_max      = 150;
map_size    = 20;
base_policy = [policy.zeros, policy.random, policy.roomba]
base_nick = ['zero', 'random', 'roomba']

In [ ]:
# Measure Control Policies
ctrl_policy = [policy.zeros, policy.random, policy.roomba]
ctrl_nick = ['zero', 'random', 'roomba']

for base_name, r_policy in zip(base_nick, base_policy):
    policy_red = r_policy.PolicyGen(env.get_map, env.get_team_red)
    
    for policy_name, b_policy in zip(ctrl_nick, ctrl_policy):
        policy_blue = b_policy.PolicyGen(env.get_map, env.get_team_red)
    
        # reset the environment and select the policies for each of the team
        observation = env.reset(map_size=map_size,
                                policy_blue=policy_blue,
                                policy_red=policy_red)
        for _ in range(n_episode):
            total_reward = 0
            for frame in range(ep_max):
                action = policy_blue.gen_action(env.get_team_blue, env._env)
                observation, reward, done, info = env.step(action)
                #observation, reward, done, info = env.step()  # feedback from environment
                #pre_score = reward;
                total_reward += reward
                if done:
                    break
            env.reset()
            